In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Subset
import numpy as np
from Models import MoELSTM
import os
from collections import OrderedDict
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader

from typing import List, Tuple, Optional, Dict
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from darts import TimeSeries
from darts.dataprocessing.transformers import Scaler
import random
from Models import MoELSTM, LSTMModel, train_model
from Preprocess import (
    compute_metrics,
    convert_timeseries_to_numpy,
    create_dataloader,
    load_building_series,
    split_series_list,
)
import pandas as pd
from collections import defaultdict
import os
import torch
import torch.optim as optim
from tqdm import tqdm


from Models import model_fn
from tqdm import tqdm
from my_utils import get_weights, set_weights, load_energy_data_feather_transformer


In [3]:
from AggregationStrategy import sync_aggregate,average_weights,sync_aggregate_norm,sync_aggregate_softmax, fedavgm_update

In [4]:
df = pd.read_feather("train_final.feather")

In [5]:
df.head()

,building_id,meter,timestamp,meter_reading,primary_use,air_temperature
7593144,0,0,2016-05-21 01:00:00,72.221012,Education,25.6
7593145,1,0,2016-05-21 01:00:00,39.611586,Education,25.6
7593146,2,0,2016-05-21 01:00:00,1.920567,Education,25.6
7593147,3,0,2016-05-21 01:00:00,111.532464,Education,25.6
7593148,4,0,2016-05-21 01:00:00,456.734799,Education,25.6


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11712248 entries, 7593144 to 20216099
Data columns (total 6 columns):
 #   Column           Dtype         
---  ------           -----         
 0   building_id      int64         
 1   meter            int64         
 2   timestamp        datetime64[ns]
 3   meter_reading    float64       
 4   primary_use      object        
 5   air_temperature  float64       
dtypes: datetime64[ns](1), float64(2), int64(2), object(1)
memory usage: 625.5+ MB


In [7]:


# Config
# List of models to experiment with
MODEL_NAMES = ["transformer"]

# Config
NUM_CLIENTS = 1410
CLIENT_FRAC = 0.15
NUM_ROUNDS = 50
LOCAL_EPOCHS = 5
LR = 0.001
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
DATA_FILE ="train_final.feather" # "meter_0_data_cleaned.feather"


In [8]:

def train_model_transformer(model, train_loader, device=None, learning_rate=0.001, loss_fn=None, optimizer_class=optim.Adam, epochs=50):
    """Train the model and return the average loss."""
    if device is None:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    model.to(device)
    loss_fn = loss_fn or nn.MSELoss()
    optimizer = optimizer_class(model.parameters(), lr=learning_rate)
    loss_history = []

    model.train()
    for epoch in range(epochs):
        epoch_loss = 0.0
        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            # if y_batch.dim() == 3 and y_batch.shape[-1] == 1:
            #     y_batch = y_batch.squeeze(-1)

            optimizer.zero_grad()
            output = model(X_batch)
            # loss = loss_fn(output, y_batch)
            loss = loss_fn(output.squeeze(-1), y_batch)  # (batch_size, 24)

            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()
        loss_history.append(epoch_loss/len(train_loader))

    # fin_loss = epoch_loss / len(train_loader)

    return get_weights(model), loss_history

## SCAFFOLD

In [8]:
def train_model_scaffold(
    local_model,
    train_loader,         # global_model, train_loader
    global_weights,    # x
    server_c,          # c
    client_ci,         # cᵢ
    device=DEVICE,
    learning_rate= LR,
    loss_fn=None,
    optimizer_class=optim.Adam,
    epochs= LOCAL_EPOCHS  # 50
):
    """Train client with SCAFFOLD correction. Return Δy, Δc, new cᵢ, final weights."""
    if device is None:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    # print(f"[DEBUG] Training on: {device}")

    
    local_model.to(device)
    loss_fn = loss_fn or nn.MSELoss()
    optimizer = optimizer_class(local_model.parameters(), lr=learning_rate)
    loss_history = []

    local_model.train()
    total_steps = 0

    for epoch in range(epochs):
        epoch_loss = 0.0
        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            # if y_batch.dim() == 3 and y_batch.shape[-1] == 1:
            #     y_batch = y_batch.squeeze(-1)

            optimizer.zero_grad()
            output = local_model(X_batch)
            loss = loss_fn(output.squeeze(-1), y_batch)
            loss.backward()
            optimizer.step()

            # ✅ SCAFFOLD correction: adjust each param after normal SGD step  # doubt at this step
            with torch.no_grad():
                for p, sc_np, ci_np in zip(local_model.parameters(), server_c, client_ci):
                    sc_tensor = torch.tensor(sc_np, dtype=p.dtype, device=p.device)
                    ci_tensor = torch.tensor(ci_np, dtype=p.dtype, device=p.device)
                    p -= learning_rate * (sc_tensor - ci_tensor)

            epoch_loss += loss.item()
            total_steps += 1
        
        loss_history.append(epoch_loss / len(train_loader))

    # Compute deltas
    local_weights = get_weights(local_model)
    delta_y = [lw - gw for lw, gw in zip(local_weights, global_weights)]

    # K = total_steps
    K = total_steps
    new_ci = []
    delta_c = []

    for gw, lw, ci, sc in zip(global_weights, local_weights, client_ci, server_c):
        ci_new = ci - sc + (gw - lw) / (K * learning_rate)
        new_ci.append(ci_new)  # doubt can overide new_ci
        delta_c.append(ci_new - ci)

    return delta_y, delta_c, new_ci, local_weights, loss_history

In [ ]:
for model_name in MODEL_NAMES:
    print(f"Starting experiment with model: {model_name}")

    # === SINGLE global model, weights, c, and all clients use it ===
    global_model = model_fn(model_name).to(DEVICE)
    state_dict = torch.load("results/transformer/transformer_round_35_scaffold_lr.pt", weights_only=True)
    global_model.load_state_dict(state_dict)
    global_weights = get_weights(global_model)
    # DEVICE Print
    print(f"Using device: {DEVICE}")

    

    server_c = [np.zeros_like(w) for w in global_weights]
    client_cs = {cid: [np.zeros_like(w) for w in global_weights] for cid in range(NUM_CLIENTS)}

    for rnd in range(35,NUM_ROUNDS):
        print(f"Round {rnd+1}/{NUM_ROUNDS}")

        sampled_clients = random.sample(range(NUM_CLIENTS), k=int(CLIENT_FRAC * NUM_CLIENTS))
        print(f"Sampled {len(sampled_clients)} clients")

        local_weight_deltas = []
        local_c_deltas = []

        for cid in tqdm(sampled_clients):
            local_model = model_fn(model_name).to(DEVICE)
            set_weights(local_model, global_weights)

            train_loader, _ = load_energy_data_feather_transformer(cid, filepath=DATA_FILE)

            delta_y, delta_c, new_ci, local_weights, fin_loss = train_model_scaffold(
                local_model, train_loader,
                global_weights = global_weights,
                server_c = server_c,
                client_ci = client_cs[cid],
                device = DEVICE,
                learning_rate = LR,
                loss_fn = None,
                optimizer_class = optim.Adam,
                epochs = LOCAL_EPOCHS
            )

            local_weight_deltas.append(delta_y)
            local_c_deltas.append(delta_c)
            client_cs[cid] = new_ci

        # === Aggregate and update global weights ===
        mean_delta_y = average_weights(local_weight_deltas)
        SERVER_LR = 1.0  # you can tune this if needed

        global_weights = [
            gw + SERVER_LR * dy for gw, dy in zip(global_weights, mean_delta_y)
        ]

        mean_delta_c = average_weights(local_c_deltas)
        frac = len(sampled_clients) / NUM_CLIENTS

        server_c = [
            sc + frac * dc for sc, dc in zip(server_c, mean_delta_c)
        ]

        set_weights(global_model, global_weights)

        ckpt_path = os.path.join("results", model_name, f"{model_name}_round_{rnd+1}_scaffold.pt")
        torch.save(global_model.state_dict(), ckpt_path)
        print(f"Saved: {ckpt_path}")


In [ ]:
# for model_name in MODEL_NAMES:
#     print(f"Starting experiment with model: {model_name}")

#     # === SINGLE global model, weights, c, and all clients use it ===
#     global_model = model_fn(model_name).to(DEVICE)
#     global_weights = get_weights(global_model)
#     # DEVICE Print
#     print(f"Using device: {DEVICE}")

    

#     server_c = [np.zeros_like(w) for w in global_weights]
#     client_cs = {cid: [np.zeros_like(w) for w in global_weights] for cid in range(NUM_CLIENTS)}

#     for rnd in range(NUM_ROUNDS):
#         print(f"Round {rnd+1}/{NUM_ROUNDS}")

#         sampled_clients = random.sample(range(NUM_CLIENTS), k=int(CLIENT_FRAC * NUM_CLIENTS))
#         print(f"Sampled {len(sampled_clients)} clients")

#         local_weight_deltas = []
#         local_c_deltas = []

#         for cid in tqdm(sampled_clients):
#             local_model = model_fn(model_name).to(DEVICE)
#             set_weights(local_model, global_weights)

#             train_loader, _ = load_energy_data_feather_transformer(cid, filepath=DATA_FILE)

#             delta_y, delta_c, new_ci, local_weights, fin_loss = train_model_scaffold(
#                 local_model, train_loader,
#                 global_weights = global_weights,
#                 server_c = server_c,
#                 client_ci = client_cs[cid],
#                 device = DEVICE,
#                 learning_rate = LR,
#                 loss_fn = None,
#                 optimizer_class = optim.Adam,
#                 epochs = LOCAL_EPOCHS
#             )

#             local_weight_deltas.append(delta_y)
#             local_c_deltas.append(delta_c)
#             client_cs[cid] = new_ci

#         # === Aggregate and update global weights ===
#         mean_delta_y = average_weights(local_weight_deltas)
#         SERVER_LR = 1.0  # you can tune this if needed

#         global_weights = [
#             gw + SERVER_LR * dy for gw, dy in zip(global_weights, mean_delta_y)
#         ]

#         mean_delta_c = average_weights(local_c_deltas)
#         frac = len(sampled_clients) / NUM_CLIENTS

#         server_c = [
#             sc + frac * dc for sc, dc in zip(server_c, mean_delta_c)
#         ]

#         set_weights(global_model, global_weights)

#         ckpt_path = os.path.join("results", model_name, f"{model_name}_round_{rnd+1}_scaffold.pt")
#         torch.save(global_model.state_dict(), ckpt_path)
#         print(f"Saved: {ckpt_path}")


Starting experiment with model: transformer
Using device: cuda
Round 1/50
Sampled 211 clients


100%|██████████| 211/211 [23:32<00:00,  6.69s/it]


Saved: results/transformer/transformer_round_1_scaffold.pt
Round 2/50
Sampled 211 clients


 19%|█▉        | 41/211 [10:19<43:46, 15.45s/it]

: 